## 1. Transformer Block

In [2]:
import torch
from torch import nn

In [3]:
# Defining the GPT configuration settings

GPT_Config_124M = {
    "vocab_size": 50257,
    "context_length": 256, 
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "dropout_rate": 0.0,
    "qkv_bias": False
}

In [4]:
# 1. Activation function
class GELUActivation(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        # Pass the input through the GELU activation - Approximate formula
        return 0.5 * x * (1 + torch.tanh(torch.sqrt(torch.tensor(2.0) / torch.pi) * (x + 0.44715 * torch.pow(x, 3))))
        
# 2. Layer normalization
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        # Defining the epsilon -  small constant added to the variance to prevent zero division - undefined - limits
        self.eps = 1e-5
        # Defining the scaling and shifting parameters - trainable - better results
        self.shift = nn.Parameter(torch.zeros(emb_dim))
        self.scale = nn.Parameter(torch.ones(emb_dim))
    # Forward pass
    def forward(self, x):
        # Getting the mean and variance of each row
        mean = x.mean(dim=-1, keepdim=True)
        variance = x.var(dim=-1, keepdim=True)
        # Getting the normalization values
        norm_x = (x - mean) / torch.sqrt(variance + self.eps)
        # Returning the normalized values of x shifted and scaled - finetuning parameters
        return self.scale * norm_x + self.shift
        
# 3. Feed forward        
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], cfg["emb_dim"]*4),
            GELUActivation(),
            nn.Linear(cfg["emb_dim"]*4, cfg["emb_dim"])
        )
    def forward(self, x):
        return self.layers(x)
# 4. Attention Mechanism
# Creating the multi-head attention compact class
import torch
from torch import nn

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, num_heads, context_length, dropout_rate, bias_units=False):
        super().__init__()
        assert d_out % num_heads == 0, "dimensions out must be divisible by number of heads"
        # Getting the head dimensions
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        # Initializing the key query value weights - (d_out, d_out)
        self.w_key = nn.Linear(d_in, d_out, bias=bias_units)
        self.w_query = nn.Linear(d_in, d_out, bias=bias_units)
        self.w_value = nn.Linear(d_in, d_out, bias=bias_units)
        # Initializing the final projection layer - optional - (d_out, d_out)
        self.out_proj = nn.Linear(d_out, d_out)
        # Creating the masking layer
        self.register_buffer("mask", torch.triu(
            torch.ones(context_length, context_length),
            diagonal = 1
        ))
        # Creating the dropout layer
        self.dropout = nn.Dropout(dropout_rate)
    # Forward pass    
    def forward(self, x):
        # Exploding the input shape
        b, num_tokens, d_out = x.shape
        # Getting the key query value matrices (b, num_tokens, d_out)
        keys = self.w_key(x)
        queries = self.w_query(x)
        values =  self.w_value(x)
        # Reshaping the key query value matrices - (b, num_tokens, num_head, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        # Grouping by number of heads - (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)
        # Getting the attention scores - (b, num_heads, num_tokens, num_tokens)
        attention_scores = queries @ keys.transpose(2, 3)
        # Masking the attention scores
        attention_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens],
            -torch.inf
        )
        # Scaling the attention scores
        attention_scores = attention_scores / keys.shape[-1]**0.5
        # Getting the attention weights
        attention_weights = torch.softmax(attention_scores, dim=-1)
        # Implementing the dropout layer
        attention_weights = self.dropout(attention_weights)
        # Getting the context vector - (b, num_heads, num_tokens, head_dim)
        context_vector = attention_weights @ values
        # Reshaping the context vectors - (b, num_tokens, num_heads, head_dim)
        context_vector = context_vector.transpose(1, 2)
        # Combining the result of mutiple heads - d_out = num_heads * head_dim
        context_vector = context_vector.contiguous().view(b, num_tokens, d_out)
        # Passing the final context vector into the projection layer - optional
        context_vector = self.out_proj(context_vector)
        return context_vector

In [5]:
# NB: Input = [b, num_tokens, 768]

class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # Defining the normalization layers
        self.layerNorm1 = LayerNorm(cfg["emb_dim"])
        self.layerNorm2 = LayerNorm(cfg["emb_dim"])
        # Defining the dropout layers
        self.drop_shortcut = nn.Dropout(cfg["dropout_rate"])
        # Defining the Multi-Head Attention layer
        self.attention = MultiHeadAttention(
            d_in = cfg["emb_dim"],
            d_out = cfg["emb_dim"],
            num_heads = cfg["n_heads"],
            context_length = cfg["context_length"],
            dropout_rate = cfg["dropout_rate"],
            bias_units = cfg["qkv_bias"]
        )
        # Deefining the feed forward layer
        self.feed_forward = FeedForward(cfg)
    def forward(self, x):
        # Shortcut connection for attention block
        shortcut = x
        x = self.layerNorm1(x)
        x = self.attention(x)
        x = self.drop_shortcut(x)
        # Add the original input the output 
        x = x +  shortcut
        # Shortcut connection for the 
        x = shortcut
        x = self.layerNorm2(x)
        x = self.feed_forward(x)
        x = self.drop_shortcut(x)
        # Add the original output
        x = x + shortcut
        return x

## 2. GPT-Model

In [6]:
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # Defining the token embedding layer
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        # Defining the positional embedding layer
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        # Defining the dropout layer 
        self.drop_emb = nn.Dropout(cfg["dropout_rate"])
        # Defining the transformer blocks
        self.transformer_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )
        # Defining the final normalization layer
        self.final_norm = LayerNorm(cfg["emb_dim"])
        # Defining the final linear layer
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)
    # Forward pass
    def forward(self, in_idx):
        # Explosion of shape
        batch_size, seq_len = in_idx.shape
        # Getting the token embeddings
        token_embeddings = self.tok_emb(in_idx)
        # Getting the positional embeddings
        positional_embeddings = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        # Getting the input embeddings
        input_embeddings = token_embeddings + positional_embeddings
        # Passing the input embeddings through the dropout layer
        input_embeddings = self.drop_emb(input_embeddings)
        # Passing the input embeddings through the transformer blocks
        input_embeddings = self.transformer_blocks(input_embeddings)
        # Passing the input embeddings through the final normalization layer
        input_embeddings = self.final_norm(input_embeddings)
        # Passint the input embeddings through the final linear layer to get logits
        logits = self.out_head(input_embeddings)
        return logits

In [7]:
# Example of instantiation

sample_batch = torch.tensor([[6109, 3636, 6100, 345],[6109, 1110, 6622, 257]])
print(sample_batch.shape)
torch.manual_seed(42)
model = GPTModel(GPT_Config_124M)
out = model(sample_batch)
print(out.shape)

torch.Size([2, 4])
torch.Size([2, 4, 50257])


In [8]:
# Checking the number of parameters

total_params = sum(p.numel() for p in model.parameters())
total_params

163009536

- We can see this is not same as the original plan of 124M instead we have 163M
- This is due to a concept weight tying
- The GPT architechts reused the weight for the token embedding layer in the output layer

In [9]:
# Confirming shapes match

model.out_head.weight.shape == model.tok_emb.weight.shape

True

In [10]:
# Confirming the theory of weight reusal

GPT_MODEL_PARAMS = total_params - sum(p.numel() for p in model.out_head.parameters())
GPT_MODEL_PARAMS

124412160

## 3.Generating output tokens

In [11]:
# Check LLM book for further clarification

def generate_tokens_simple(model, idx, max_num_tokens, context_size):
    # Number of iterations - number of new tokens to be generated
    for _ in range(max_num_tokens):
        # idx.shape = [batch, num_tokens] e.g = [2, 2]
        # Slicing - making sure the context size = 5 if not select last 5 tokens
        idx_condition = idx[:, -5:]
        # Making predictions - getting the logits - logits.shape = [b, num_tokens, vocab_size]
        with torch.no_grad():
            logits = model(idx_condition)
        # Extracting the last token row - logits[:, -1, :].shape = [b, 1, vocab_size]
        logits = logits[:, -1, :]
        # Converting the logits into probabilities - exponential sum
        probabilities = torch.softmax(logits, dim=-1)
        # Getting the element with the highest probability
        idx_next = torch.argmax(probabilities, dim=-1, keepdim=True)
        # Appending the predicted token to the running sequence
        # idx = [b, num_tokens+1]
        idx = torch.cat((idx, idx_next), dim=1)
    # Returning the full sequence = original + generated upto max number of tokens
    return idx   

## Testing Generation using real text

In [12]:
# Encoding text to token ids
import tiktoken

starting_context = "Hello, I am Jacob from Congo"
tokenizer = tiktoken.get_encoding("gpt2")
encoded = tokenizer.encode(starting_context)
print("Encoded text = ", encoded)
# Converting the encode into a tensor
# The unsqueeze - we are adding an dimension: [num_tokens] ---> [b, num_tokens]
encoded = torch.tensor(encoded).unsqueeze(0)
print("Encoded tensor = ", encoded, "Shape = ", encoded.shape)

Encoded text =  [15496, 11, 314, 716, 12806, 422, 25935]
Encoded tensor =  tensor([[15496,    11,   314,   716, 12806,   422, 25935]]) Shape =  torch.Size([1, 7])


In [13]:
# Generating text using our model

# Setting the evaluation/testing mode - cancels some unnecessary components e.g dropout layers
model.eval()
output_tensor = generate_tokens_simple(
    model = model,
    idx = encoded,
    max_num_tokens = 6,
    context_size = GPT_Config_124M["context_length"]
)
print(f"Output tensor = {output_tensor} Output tensor shape = {output_tensor.shape}")

Output tensor = tensor([[15496,    11,   314,   716, 12806,   422, 25935,  8559, 24744, 13606,
         19558, 22782, 30214]]) Output tensor shape = torch.Size([1, 13])


In [14]:
# Decoding the output tensor - remember we need to squeeze the first batch dimension
# Note that the tiktoken doesnt work with tensor so use python lists or numpy arrays

decoded_text = tokenizer.decode(output_tensor.squeeze(0).numpy())
decoded_text

'Hello, I am Jacob from Congo Fredrots Marshall fog punkFri'